In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout
from tensorflow.keras.applications import imagenet_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import os
import glob
import shutil
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
from PIL import Image, ImageFile
import zipfile
from tensorflow.keras.utils import Sequence
from imblearn.over_sampling import RandomOverSampler
from imblearn.tensorflow import balanced_batch_generator

In [9]:
# get list of all the classes
path = 'dataset/train'
list_files = os.listdir(path)
list_files

['Abalistes_stellatus',
 'Abudefduf_saxatilis',
 'Acanthemblemaria_spinosa',
 'Acanthochromis_polyacanthus',
 'Acanthurus_achilles',
 'Acanthurus_chirurgus',
 'Acanthurus_coeruleus',
 'Acanthurus_dussumieri',
 'Acanthurus_japonicus',
 'Acanthurus_leucosternon',
 'Acanthurus_lineatus',
 'Acanthurus_maculiceps',
 'Acanthurus_nigricans',
 'Acanthurus_nigrofuscus',
 'Acanthurus_nigroris',
 'Acanthurus_olivaceus',
 'Acanthurus_pyroferus',
 'Acanthurus_sohal',
 'Acanthurus_tennenti',
 'Acanthurus_thompsoni',
 'Acanthurus_triostegus',
 'Acanthurus_tristis',
 'Acanthurus_xanthopterus',
 'Aetobatus_narinari',
 'Alectis_indicus',
 'Amblyapistus_taenianotus',
 'Amblycirrhitus_pinos',
 'Amblyeleotris_diagonalis',
 'Amblyeleotris_guttata',
 'Amblyeleotris_randalli',
 'Amblyeleotris_steinitzi',
 'Amblyeleotris_wheeleri',
 'Amblyglyphidodon_aureus',
 'Amblygobius_decussatus',
 'Amblygobius_hectori',
 'Amblygobius_phalaena',
 'Amblygobius_rainfordi',
 'Amphiprion_clarkii',
 'Amphiprion_percula+Amphipr

In [10]:
train_dir = 'dataset/train/'
valid_dir = 'dataset/valid/'
test_dir = 'dataset/test/'

In [5]:
# valid base generator
val_gen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)

# train base generator with augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [6]:
# train and valid generator
train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

validation_generator = val_gen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 30057 images belonging to 549 classes.
Found 8667 images belonging to 549 classes.


In [11]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [8]:
# oversampling generator class
class OversamplingDataGenerator:
    def __init__(self, generator, oversampler):
        self.generator = generator
        self.oversampler = oversampler
        self.class_indices = generator.class_indices
        self.samples = generator.samples
        self.batch_size = generator.batch_size
        self.image_shape = generator.image_shape
        self.num_classes = len(generator.class_indices)

    def __iter__(self):
        return self

    def __next__(self):
        x_batch, y_batch = next(self.generator)
        y_batch_flat = np.argmax(y_batch, axis=1)
        x_batch_resampled, y_batch_resampled = self.oversampler.fit_resample(x_batch.reshape((x_batch.shape[0], -1)), y_batch_flat)
        x_batch_resampled = x_batch_resampled.reshape((-1,) + self.image_shape)
        y_batch_resampled = np.eye(self.num_classes)[y_batch_resampled]

        return x_batch_resampled, y_batch_resampled

    def __len__(self):
        return len(self.generator)

In [9]:
# make oversampler and set to minority (make small amount to large amount)
oversampler = RandomOverSampler(sampling_strategy='minority')
# oversampling on train generator
oversampled_train_generator = OversamplingDataGenerator(train_generator, oversampler)

In [12]:
# test generator
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical'
)

Found 4518 images belonging to 549 classes.


In [11]:
# show number of classes and list of classes
num_classes = len(oversampled_train_generator.class_indices)
class_labels = list(oversampled_train_generator.class_indices.keys())
print('Class labels:', class_labels)
print('Number of classes:', num_classes)

Class labels: ['Abalistes_stellatus', 'Abudefduf_saxatilis', 'Acanthemblemaria_spinosa', 'Acanthochromis_polyacanthus', 'Acanthurus_achilles', 'Acanthurus_chirurgus', 'Acanthurus_coeruleus', 'Acanthurus_dussumieri', 'Acanthurus_japonicus', 'Acanthurus_leucosternon', 'Acanthurus_lineatus', 'Acanthurus_maculiceps', 'Acanthurus_nigricans', 'Acanthurus_nigrofuscus', 'Acanthurus_nigroris', 'Acanthurus_olivaceus', 'Acanthurus_pyroferus', 'Acanthurus_sohal', 'Acanthurus_tennenti', 'Acanthurus_thompsoni', 'Acanthurus_triostegus', 'Acanthurus_tristis', 'Acanthurus_xanthopterus', 'Aetobatus_narinari', 'Alectis_indicus', 'Amblyapistus_taenianotus', 'Amblycirrhitus_pinos', 'Amblyeleotris_diagonalis', 'Amblyeleotris_guttata', 'Amblyeleotris_randalli', 'Amblyeleotris_steinitzi', 'Amblyeleotris_wheeleri', 'Amblyglyphidodon_aureus', 'Amblygobius_decussatus', 'Amblygobius_hectori', 'Amblygobius_phalaena', 'Amblygobius_rainfordi', 'Amphiprion_clarkii', 'Amphiprion_percula+Amphiprion_ocellaris', 'Amphipr

In [12]:
# get MobileNet base model
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [13]:
# get the 5th from the last layer ( conv_pw_13_relu (ReLU) )
x = mobile.layers[-5].output
# reshape that layer
x = tf.keras.layers.Reshape(target_shape=(1024,))(x)
# add dropout
x = Dropout(0.2)(x)
# make prediction layer
output = Dense(units=549, activation='softmax')(x)

In [14]:
# freeze base model
for layer in mobile.layers:
    layer.trainable = False

In [15]:
# get input layer from base model and combine with the output we made to make a new model
model = Model(inputs=mobile.input, outputs=output)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# earlystopping
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
         ModelCheckpoint(filepath='model/model_group_mobile_04', monitor='val_loss', save_best_only=True)]

In [19]:
model.fit(x=oversampled_train_generator,
            steps_per_epoch=len(oversampled_train_generator),
            validation_data=validation_generator,
            validation_steps=len(validation_generator),
            epochs=20,
            verbose=1,
            callbacks=callback
)

Epoch 1/20
 55/470 [==>...........................] - ETA: 15:03 - loss: 6.8705 - accuracy: 0.0020

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


307/470 [==================>...........] - ETA: 6:01 - loss: 6.3170 - accuracy: 0.0107

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


423/470 [==========================>...] - ETA: 1:44 - loss: 6.1457 - accuracy: 0.0181

...

228/470 [=============>................] - ETA: 8:49 - loss: 1.2417 - accuracy: 0.7436

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


470/470 [==============================] - ETA: 0s - loss: 1.2310 - accuracy: 0.7433

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


INFO:tensorflow:Assets written to: model\model_group_mobile_04\assets
470/470 [==============================] - 1234s 3s/step - loss: 1.2310 - accuracy: 0.7433 - val_loss: 1.2615 - val_accuracy: 0.7259


In [20]:
model.save('model/model_group_mobile_04/before_ft')

INFO:tensorflow:Assets written to: model/model_group_mobile_04/before_ft\assets


In [21]:
# finetune
# unfreeze the base model
for layer in mobile.layers:
    layer.trainable = True

In [22]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
         ModelCheckpoint(filepath='model/model_group_mobile_04/ft10+5', monitor='val_loss', save_best_only=True)]

In [ ]:
# finetune 15 epochs 

In [27]:
model.fit(x=oversampled_train_generator,
            steps_per_epoch=len(oversampled_train_generator),
            validation_data=validation_generator,
            validation_steps=len(validation_generator),
            epochs=5,
            verbose=1,
            callbacks=callback
)

Epoch 1/5
163/470 [=========>....................] - ETA: 19:34 - loss: 0.6360 - accuracy: 0.8523

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


221/470 [=============>................] - ETA: 15:54 - loss: 0.6247 - accuracy: 0.8551

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


276/470 [================>.............] - ETA: 12:22 - loss: 0.6266 - accuracy: 0.8542

...

369/470 [======================>.......] - ETA: 6:18 - loss: 0.5399 - accuracy: 0.8772

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


470/470 [==============================] - ETA: 0s - loss: 0.5431 - accuracy: 0.8767

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


INFO:tensorflow:Assets written to: model/model_group_mobile_04\ft10+5\assets
470/470 [==============================] - 1959s 4s/step - loss: 0.5431 - accuracy: 0.8767 - val_loss: 0.7570 - val_accuracy: 0.8157


In [ ]:
# fintune more 5 epochs

In [30]:
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
         ModelCheckpoint(filepath='model/model_group_mobile_04/ft10+5+5', monitor='val_loss', save_best_only=True)]

In [32]:
model.fit(x=oversampled_train_generator,
            steps_per_epoch=len(oversampled_train_generator),
            validation_data=validation_generator,
            validation_steps=len(validation_generator),
            epochs=5,
            verbose=1,
            callbacks=callback
)

Epoch 1/5
 61/470 [==>...........................] - ETA: 27:19 - loss: 0.5475 - accuracy: 0.8729

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


377/470 [=======================>......] - ETA: 6:33 - loss: 0.5247 - accuracy: 0.8785

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


397/470 [========================>.....] - ETA: 5:09 - loss: 0.5249 - accuracy: 0.8787

...

392/470 [========================>.....] - ETA: 5:11 - loss: 0.4731 - accuracy: 0.8905

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


470/470 [==============================] - ETA: 0s - loss: 0.4733 - accuracy: 0.8903

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


470/470 [==============================] - 2067s 4s/step - loss: 0.4733 - accuracy: 0.8903 - val_loss: 0.7176 - val_accuracy: 0.8246


In [ ]:
# ft 15 epochs : loss: 0.5431 - accuracy: 0.8767 - val_loss: 0.7570 - val_accuracy: 0.8157
# ft more 5 epochs end at 4 : loss: 0.4784 - accuracy: 0.8902 - val_loss: 0.7166 - val_accuracy: 0.8267

In [2]:
# load the model
model = tf.keras.models.load_model('model/model_group_mobile_04/ft10+5+4')

In [ ]:
# make predictions
predictions = model.predict(x=test_generator, steps=len(test_generator), verbose=1)

In [7]:
# evaluate the model on test set
results = model.evaluate(test_generator, verbose=1)

14/72 [====>.........................] - ETA: 1:43 - loss: 0.6630 - accuracy: 0.8471

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


72/72 [==============================] - 132s 2s/step - loss: 0.6596 - accuracy: 0.8330


In [51]:
# make predictions and get true - predicted labels
true_labels = []
predicted_labels = []

for i in range(len(test_generator)):
    batch_data, batch_labels = test_generator[i]
    true_labels.extend(batch_labels)
    predicted_labels.extend(model.predict(batch_data))

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

In [52]:
# get the class(labels) no.
true_labels_int = np.argmax(true_labels, axis=1)
predicted_labels_int = np.argmax(predicted_labels, axis=1)

In [53]:
# confusion matrix
conf_matrix = confusion_matrix(true_labels_int, predicted_labels_int)

print(conf_matrix)

[[ 9  0  0 ...  0  0  0]
 [ 0 11  0 ...  0  0  0]
 [ 0  0 10 ...  0  0  0]
 ...
 [ 0  0  0 ...  8  0  0]
 [ 0  0  0 ...  0  7  0]
 [ 0  0  0 ...  0  0  9]]


In [54]:
# save the confusion matrix as csv
f = open('model/model_eva_result/model_04/confusion_matrix04.csv', 'w')
np.savetxt(filename, conf_matrix, fmt='%d', delimiter=',')
f.close()

print(f"Confusion matrix saved to {filename}")

Confusion matrix saved to model/model_eva_result/model_04/confusion_matrix04.csv


In [55]:
print(classification_report(true_labels_int,predicted_labels_int))

              precision    recall  f1-score   support

           0       0.69      1.00      0.82         9
           1       0.92      1.00      0.96        11
           2       0.59      0.91      0.71        11
           3       0.56      0.90      0.69        10
           4       1.00      0.92      0.96        12
           5       0.89      0.80      0.84        10
           6       0.71      1.00      0.83        10
           7       0.83      0.83      0.83        12
           8       0.91      0.83      0.87        12
           9       0.92      1.00      0.96        12
          10       0.92      1.00      0.96        12
          11       1.00      0.78      0.88         9
          12       0.80      0.80      0.80        10
          13       0.62      0.56      0.59         9
          14       0.43      0.43      0.43         7
          15       0.73      0.80      0.76        10
          16       0.86      0.55      0.67        11
          17       1.00    

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [56]:
# save the classification report as txt
f = open('model/model_eva_result/model_04/class_report04.txt', 'w')
f.write(classification_report(true_labels_int,predicted_labels_int))
f.close()

C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\chonl\anaconda3\envs\py39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
